<a href="https://colab.research.google.com/github/tgeorgy/rapid-detector/blob/master/examples/rapid_detector_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Cloning the rapid-detector repository...")
!git clone https://github.com/tgeorgy/rapid-detector.git

In [ ]:
print("Installing dependencies...")
!cd rapid-detector/ && pip install -e .

In [ ]:
print('Uninstalling numpy...')
!pip uninstall -y numpy
print('Installing numpy==1.26.4...')
!pip install numpy==1.26.4

In [ ]:
print('Logging in to Hugging Face Hub (this will prompt for your token)...')
from huggingface_hub import login
login()

In [ ]:
print("Starting the server...")
import sys
sys.path.append('rapid-detector')
import threading
import uvicorn

def run_server():
    try:
        from app.api import app
        # host 0.0.0.0 is required for Colab's proxy to reach the service
        uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning")
    except Exception as e:
        print(f"❌ Error: {e}")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

In [ ]:
print("Waiting for the server to start...")
import time
time.sleep(20)

1. Name Object class
2. Upload Images
3. Check detections
4. Annotate a few examples
5. Click "Use API" and copy the detector_id, use it bellow

In [ ]:
print("Opening the app UI...")
from google.colab import output
output.serve_kernel_port_as_iframe(8000, path='/', height=600)

In [ ]:
print("Coppying uploaded images...")
!cp -r ~/.cache/rapid_detector/images .

In [ ]:
print("Running inference...")
import glob
from matplotlib import pyplot
import requests

detector_id = "<YOUR DETECTOR ID FROM USE API WINDOW>"
api_url = f"http://localhost:8000/detect/{detector_id}"

for fpath in glob.glob('./images/*'):
    with open(fpath, "rb") as f:
        response = requests.post(api_url, files={"image": f})

    results = response.json()
    print(f"Found {len(results['boxes'])} detections")
    pyplot.imshow(pyplot.imread(fpath))
    for x0, y0, x1, y1 in results['boxes']:
        pyplot.plot([x0, x1, x1, x0, x0], [y0, y0, y1, y1, y0])
    pyplot.axis('off')
    pyplot.show()